## Imports

In [ ]:
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from scipy.linalg import svd
import sklearn.cluster as cluster
import plotly.express as px

np.random.seed(1234)

## 7.2 Something

### Data Load - California Housing Data Set

In [ ]:
housing = datasets.fetch_california_housing()
X = pd.DataFrame(data=housing.data, columns=housing.feature_names)
X = X.head(500)
display(X)